In [ ]:
#My VS Code has issues connecting to the venv.
# I find that first connecting to one of my conda environments
# and then switching to the venv works.
# I run this cell on conda environment first then switch to venv
# and run it there and seems to be all good.
print("Hello")

In [ ]:
# This cell allows hot reloading of modules.
# It is useful for development, so you don't have to restart the kernel.
# Some changes such as adding new files or changing the structure of the code
# may not be picked up and will require a kernel restart.
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import requests
import asyncio
import time
from dotenv import load_dotenv, find_dotenv
import logging

# Set up logging include logger name
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

#import the KServer module from the pykaahma_linz package
from pykaahma_linz import KServer

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
api_key = os.getenv('LINZ_API_KEY')

#layers
rail_station_layer_id = "50318" #rail station 175 points
fences_layer_id = "50268" #NZ Fence Centrelines  
geodetic_marks_layer_id = "50787" # NZ Geodetic Marks 132,966 point features
nz_walking_biking_tracks_layer_id = "52100" #NZ Walking and Biking Tracks 29,187 polyline features

#tables
nz_building_age_table_id = "105617" #NZ Building Age 18 records, no changeset
suburb_locality_table_id = "113761" #NZ Suburb Locality 3190 records


## Server object  
Get a reference to the LINZ server  

In [ ]:
linz = KServer.KServer(api_key)
print(linz)

## Layer item  
Run some tests on a layer item  

In [ ]:
itm = linz.content.get(rail_station_layer_id)
print(itm)
print(itm.title)

In [ ]:
#Query using WFS
res = itm.query(count=2)
res.head()

In [ ]:
# Get a changeset from the layer using WFS
res = itm.get_changeset(from_time="2024-01-01T00:00:00Z")
logging.info(f"Total records returned {itm.title}: {res.shape[0]}")
res.head()


In [ ]:
waikato_polygon = {
        "coordinates": [
          [
            [
              174.30400216373914,
              -36.87399457472202
            ],
            [
              174.30400216373914,
              -38.83764306196984
            ],
            [
              176.83017911725346,
              -38.83764306196984
            ],
            [
              176.83017911725346,
              -36.87399457472202
            ],
            [
              174.30400216373914,
              -36.87399457472202
            ]
          ]
        ],
        "type": "Polygon"
      }

In [ ]:
job = itm.export("geodatabase", crs="EPSG:2193", extent=waikato_polygon,)
#job = itm.export("geodatabase")

In [ ]:
job.status

In [ ]:
output_folder = r"c:\temp\data"
job.download(folder=output_folder)

## Table item  
Run some tests on a table item

In [ ]:
table_itm = linz.content.get(suburb_locality_table_id)
print(table_itm)
print(table_itm.title)
print(f'{table_itm.supports_changesets=}')

In [ ]:
#Query using WFS
data = table_itm.query(count=2)
data.head()

In [ ]:
# Get a changeset from the layer using WFS
cs_data = table_itm.get_changeset(from_time="2024-01-01T00:00:00Z")
logging.info(f"Total records returned {table_itm.title}: {cs_data.shape[0]}")
cs_data.head()

In [ ]:
job = table_itm.export("geodatabase")

In [ ]:
print(job)

In [ ]:
output_folder = r"c:\temp\data"
job.download(folder=output_folder)

## Test multidownload samples  
Test downloading two jobs in a sync and async fashion.  

In [ ]:
async def run_export(itm, export_format, crs, output_folder):
    job = itm.export(export_format, crs=crs)
    print(f"Started export job {job.id}")
    file_path = await job.download_async(output_folder)
    print(f"{job.id} downloaded to {file_path}")
    return file_path

async def export_multiple_items_async():
    output_folder = r"c:\temp\data\async"

    # Launch both jobs concurrently
    # print out the time taken to run both jobs
    logging.info("Starting multiple export jobs concurrently...")
    start_time = asyncio.get_event_loop().time()
    results = await asyncio.gather(
        run_export(itm, "geodatabase", "EPSG:2193", output_folder),
        run_export(itm2, "geodatabase", "EPSG:2193", output_folder),
    )
    end_time = asyncio.get_event_loop().time()
    logging.info(f"Both export jobs completed in {end_time - start_time:.2f} seconds")

    logging.info(f"Both exports completed: {str(results)}")

# In Jupyter or async context, use:
await export_multiple_items_async()

# In script:
# asyncio.run(export_multiple_jobs())

In [ ]:
def run_export_sync(itm, export_format, crs, output_folder):
    job = itm.export(export_format, crs=crs)
    print(f"Started export job {job.id}")
    file_path = job.download(output_folder)
    print(f"{job.id} downloaded to {file_path}")
    return file_path

def export_multiple_items_sync():
    output_folder = r"c:\temp\data\sync"

    logging.info("Starting multiple export jobs synchronously...")
    start_time = time.time()
    result1 = run_export_sync(itm, "geodatabase", "EPSG:2193", output_folder)
    result2 = run_export_sync(itm2, "geodatabase", "EPSG:2193", output_folder)
    end_time = time.time()
    logging.info(f"Both export jobs completed in {end_time - start_time:.2f} seconds")

    print(f"Both exports completed: {str([result1, result2])}")

# Call main_sync() in a normal script or notebook cell
export_multiple_items_sync()